In [1]:

!mkdir ~/.kaggle

In [2]:
!cp kaggle.json ~/.kaggle

In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d sunnysai12345/news-summary

 46% 9.00M/19.8M [00:00<00:00, 56.9MB/s]
100% 19.8M/19.8M [00:00<00:00, 107MB/s] 


In [5]:
!unzip news-summary.zip

Archive:  news-summary.zip
  inflating: news_summary.csv        
  inflating: news_summary_more.csv   


In [6]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 6.9 MB/s 


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 32.7 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import time
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
def set_seeds(SEED):
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

In [11]:
SEED = 42
set_seeds(SEED)

In [12]:
df = pd.read_csv('news_summary.csv', encoding="latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [13]:
print(f"Sample:\n"
    f"News: {df.iloc[10]['ctext']} \n"
    f"Summary: {df.iloc[10]['text']} \n"
    f"Headline: {df.iloc[10]['headlines']}")

Sample:
News: The Food Safety and Standards Authority of India (FSSAI) is in the process of creating a network of food banking partners to collect and distribute leftover food from large parties and weddings to the hungry.A notification to create a separate category of food business operators (FBOs), who will be licensed to deal only with leftover food, has been drafted to ensure the quality of food.?We are looking at partnering with NGOs or organisations that collect, store and distribute surplus food to ensure they maintain certain hygiene and health standards when handling food,? said Pawan Agarwal, CEO of FSSAI.?Tonnes of food is wasted annually. We are looking at creating a mechanism through which food can be collected from restaurants, weddings, large-scale parties,?  says Pawan Agarwal, ?All food, whether it is paid for or distributed free, must meet the country?s food safety and hygiene standards,? he said.The organisations in the business of collecting leftover food will now h

In [14]:
df = df[['text', 'ctext']]
df.ctext = "summarize: " + df.ctext
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,text,ctext
0,The Administration of Union Territory Daman an...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize: Lashkar-e-Taiba's Kashmir commander...
4,Hotels in Maharashtra will train their staff t...,summarize: Hotels in Mumbai and other Indian c...


In [26]:
train_size = 0.8

train_df = df.sample(frac=train_size, random_state=42).reset_index(drop=True)
valid_df = df.drop(train_df.index).reset_index(drop=True)

In [27]:
MODEL_NAME = "t5-base"
MODEL_PATH = "model.pt"
MAX_LEN = 512
TOKENIZER = T5Tokenizer.from_pretrained(MODEL_NAME, model_max_length=MAX_LEN)
SUMMARY_LEN = 150
BATCH_SIZE = 2 
EPOCHS = 4
LR = 1e-4

In [42]:
class NewsDataset(Dataset):
    def __init__(self, df, tokenizer, source_len, summary_len):
        super().__init__()

        self.tokenizer = tokenizer
        self.data = df
        self.source_len = source_len
        self.summary_len = summary_len
        self.text = df.text
        self.ctext = df.ctext
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item):
        ctext = str(self.ctext[item])
        ctext = " ".join(ctext.split())

        text = str(self.text[item])
        text = " ".join(text.split())

        source = self.tokenizer.encode_plus(
            text,
            max_length=self.source_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt')
        
        target = self.tokenizer.encode_plus(
            ctext,
            max_length=self.summary_len,
            padding='max_length',
            return_attention_mask=True,
            truncation=True,
            return_tensors='pt')

        return {
            "source_ids": source["input_ids"].flatten(),
            "source_mask": source["attention_mask"].flatten(),
            "target_ids": target["input_ids"].flatten(),
            "target_mask": target["attention_mask"].flatten()
        }

In [43]:
train_dataset = NewsDataset(train_df, TOKENIZER, MAX_LEN, SUMMARY_LEN)
valid_dataset = NewsDataset(valid_df, TOKENIZER, MAX_LEN, SUMMARY_LEN)

In [44]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, pin_memory=True)
val_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, pin_memory=True)

In [45]:

# sample check
sample = next(iter(train_dataloader))
sample['source_ids'].shape, sample['source_mask'].shape, sample['target_ids'].shape

(torch.Size([2, 512]), torch.Size([2, 512]), torch.Size([2, 150]))

In [46]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to(device)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [47]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)

In [48]:
def train(dataloader, model, tokenizer, optimizer, device):
    model.train()

    total_steps = len(dataloader)
    epoch_loss = 0

    for idx, batch in enumerate(dataloader):
        ids = batch["source_ids"].to(device)
        mask = batch["source_mask"].to(device)

        target_ids = batch["target_ids"].to(device)
        
        y_ids = target_ids[:, :-1].contiguous()
        lm_labels = target_ids[:, 1:].clone().detach()
        lm_labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

        optimizer.zero_grad()

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels
        )

        loss = outputs[0]
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

        if idx%1000 == 0:
            print(f"Step: {idx}/{total_steps} | Loss: {loss.item()}")
    
    return epoch_loss / total_steps

In [49]:
def evaluate(dataloader, model, tokenizer, device):
    model.eval()

    total_steps = len(dataloader)
    epoch_loss = 0

    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            ids = batch["source_ids"].to(device)
            mask = batch["source_mask"].to(device)

            target_ids = batch["target_ids"].to(device)
            
            y_ids = target_ids[:, :-1].contiguous()
            lm_labels = target_ids[:, 1:].clone().detach()
            lm_labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            outputs = model(
                input_ids=ids,
                attention_mask=mask,
                decoder_input_ids=y_ids,
                labels=lm_labels
            )

            loss = outputs[0]
            epoch_loss += loss.item()

            if idx%1000 == 0:
                print(f"Val Step: {idx}/{total_steps} | Loss: {loss.item()}")
    
    return epoch_loss / total_steps

In [50]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs 

In [51]:
best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    start_time = time.time()
    train_loss = train(train_dataloader, model, TOKENIZER, optimizer, device)
    val_loss = evaluate(val_dataloader, model, TOKENIZER, device)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), MODEL_PATH)
    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(f"\t Train Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):5.4f}")
    print(f"\t Val Loss: {val_loss:.3f} | Val PPL: {np.exp(val_loss):5.4f}")

Step: 0/1806 | Loss: 11.934693336486816
Step: 1000/1806 | Loss: 3.1119415760040283
Val Step: 0/452 | Loss: 2.095304012298584
Epoch: 01 | Epoch Time: 12m 59s
	 Train Loss: 2.867 | Train PPL: 17.5863
	 Val Loss: 2.284 | Val PPL: 9.8197
Step: 0/1806 | Loss: 3.007807493209839
Step: 1000/1806 | Loss: 2.8161847591400146
Val Step: 0/452 | Loss: 1.9069793224334717
Epoch: 02 | Epoch Time: 12m 58s
	 Train Loss: 2.533 | Train PPL: 12.5920
	 Val Loss: 2.086 | Val PPL: 8.0528
Step: 0/1806 | Loss: 2.797226667404175
Step: 1000/1806 | Loss: 2.6095829010009766
Val Step: 0/452 | Loss: 1.7501214742660522
Epoch: 03 | Epoch Time: 12m 57s
	 Train Loss: 2.351 | Train PPL: 10.4977
	 Val Loss: 1.934 | Val PPL: 6.9192
Step: 0/1806 | Loss: 2.603055477142334
Step: 1000/1806 | Loss: 2.476139783859253
Val Step: 0/452 | Loss: 1.5998451709747314
Epoch: 04 | Epoch Time: 12m 57s
	 Train Loss: 2.194 | Train PPL: 8.9736
	 Val Loss: 1.806 | Val PPL: 6.0841


In [55]:
def inference(model, news, tokenizer, device):
    model.eval()

    news = "summarize: " + news

    source = tokenizer.encode_plus(
        news,
        max_length=MAX_LEN,
        padding="max_length",
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt')
    
    with torch.no_grad():
        ids = source["input_ids"].to(device)
        mask = source["attention_mask"].to(device)

        generated_ids = model.generate(
            input_ids=ids,
            attention_mask=mask,
            max_length=SUMMARY_LEN,
            num_beams=2,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True
        )

        summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

    return summary[0]

In [56]:
news = "The Food Safety and Standards Authority of India (FSSAI) is in the process of creating a network of food banking partners to collect and distribute leftover food from large parties and weddings to the hungry.A notification to create a separate category of food business operators (FBOs), who will be licensed to deal only with leftover food, has been drafted to ensure the quality of food.?We are looking at partnering with NGOs or organisations that collect, store and distribute surplus food to ensure they maintain certain hygiene and health standards when handling food,? said Pawan Agarwal, CEO of FSSAI.?Tonnes of food is wasted annually. We are looking at creating a mechanism through which food can be collected from restaurants, weddings, large-scale parties,?  says Pawan Agarwal, ?All food, whether it is paid for or distributed free, must meet the country?s food safety and hygiene standards,? he said.The organisations in the business of collecting leftover food will now have to work in collaboration with FSSAI so their efforts can be scaled up.?Tonnes of food is wasted annually and can be used to feed several thousands. We are looking at creating a mechanism through which food can be collected from restaurants, weddings, large-scale parties etc,? said Agarwal.The initiative will set up a helpline network where organisations can call in for collection but reaching individuals who want to directly donate food will take time. ?We will have a central helpline number. Reaching people at the household level may not be feasible initially but it is an integral part of the long-term plan,? he said. ?We have begun collecting names of people working in the sector. There are still a few months to go before the scheme materialises,? said Agarwal.?Collecting food going waste to feed the hungry is a noble thought but to transport, store and maintain the cold chain of cooked food is a huge challenge. The logistics are a nightmare, which is why we don?t handle leftovers and only distribute uncooked food that can be cooked locally,? said Kuldip Nar, founder of Delhi NCR Food Bank, which has been feeding the poor in 10 cities since 2011."

In [57]:
news = " ".join(news.split()).strip()
summary = inference(model, news, TOKENIZER, device)
print(summary)

The Food Safety and Standards Authority of India (FSSAI) is in the process of creating a network of food banking partners to collect and distribute leftover food from large parties and weddings to the hungry.The initiative will set up a helpline network where organisations can call in for collection but reaching individuals who want to directly donate food will take time, says Pawan Agarwal, CEO of FSSAI.?We are looking at partnering with NGOs or organisations that collect, store and distribute surplus food to ensure they maintain certain hygiene and health standards when handling food,? said Agarwal.
